# Attention Architecture

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import music21

In [3]:
# from fastai.text import *
from enum import Enum
import torch
from fastai.text.models.awd_lstm import *
from fastai.text.models.transformer import *

In [4]:
import numpy as np
import torch.nn as nn

In [5]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [6]:
import sys
sys.path.insert(0, '../../')
from src.fastai_data import *
from src.encode_data import *
from src.serve import *

In [7]:
from src.music_transformer import *

In [8]:
path = Path('../../data/midi/v15/piano_duet/')

In [40]:
from fastai.text.models.transformer import _line_shift

## Single Stream Encoding

In [9]:
config = v15s_config(vocab); config

{'ctx_len': 150,
 'n_layers': 16,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 512,
 'mask': True,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 16,
 'bptt': 256,
 'vocab_size': 274}

## Fastai Learner

In [10]:
dl_tfms = [mask_tfm, next_sentence_tfm]

In [11]:
data = load_music_data(path, cache_name='tmp/sample', vocab=vocab, y_offset=0, dl_tfms=dl_tfms, **config)

In [12]:
xb,yb = data.one_batch(cpu=False)

In [13]:
xb

tensor([[155,  73, 155,  ...,   8, 147,  51],
        [155,  60, 155,  ...,   8, 147,   4],
        [155,  68, 155,  ...,   8, 147,   4],
        ...,
        [155,  68, 155,  ...,   8, 147,  52],
        [155,  68, 155,  ...,   8, 147,  54],
        [155,  73,   4,  ...,   8,   4,  51]], device='cuda:0')

In [14]:
yb

[tensor([[  1,   1,   1,  ...,   1,   1,   1],
         [  1,  68,   1,  ...,   1,   1,  52],
         [  1,   1,   1,  ...,   1,   1,  52],
         ...,
         [155,   1,   1,  ...,   1,   1,   1],
         [  1,   1, 155,  ...,   1,   1,   1],
         [  1,   1, 155,  ...,   1, 147,   1]], device='cuda:0'),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')]

## LMNP

In [15]:
# m_len = 0
# x_len = 16 # bptt
# seq_len = m_len+x_len
# torch.triu(torch.ones(x_len, seq_len), diagonal=m_len).byte()[None,None].cpu().numpy()
# torch.triu(torch.ones(x_len, seq_len), diagonal=m_len+1).byte()[None,None].cpu().numpy()

In [16]:
import torch.nn as nn

In [17]:
class TransformerEmbedding(nn.Module):
    "Embedding + positional encoding + dropout"
    def __init__(self, vocab_sz:int, emb_sz:int, inp_p:float=0.):
        super().__init__()
        self.emb_sz = emb_sz
        self.embed = embedding(vocab_sz, emb_sz)
        self.pos_enc = PositionalEncoding(emb_sz)
        self.drop = nn.Dropout(inp_p)
    
    def forward(self, inp):
        pos = torch.arange(0, inp.size(1), device=inp.device).float()
        return self.drop(self.embed(inp)), self.pos_enc(pos)

In [18]:
class MusicTransformer(nn.Module):
    "TransformerXL model: https://arxiv.org/abs/1901.02860."
    def __init__(self, embed:nn.Module, n_layers:int, n_heads:int, d_model:int, d_head:int, d_inner:int, 
                 resid_p:float=0., attn_p:float=0., ff_p:float=0., bias:bool=False, scale:bool=True,
                 act:Activation=Activation.ReLU, double_drop:bool=True, attn_cls:Callable=MultiHeadRelativeAttention,
                 learned_pos_enc:bool=False, mask:bool=True, **kwargs):
        super().__init__()
        self.encoder = embed
        self.u = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.v = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.n_layers,self.d_model,self.mask = n_layers,d_model,mask
        self.layers = nn.ModuleList([DecoderLayer(n_heads, d_model, d_head, d_inner, resid_p=resid_p, attn_p=attn_p,
                      ff_p=ff_p, bias=bias, scale=scale, act=act, double_drop=double_drop, 
                      attn_cls=attn_cls) for k in range(n_layers)])
        
        nn.init.normal_(self.u, 0., 0.02)
        nn.init.normal_(self.v, 0., 0.02)
    
    def forward(self, x):
        bs,x_len = x.size()
        inp, pos_enc = self.encoder(x)

        mask = window_mask(x_len, x.device) if self.mask else None
        
        for i, layer in enumerate(self.layers):
            inp = layer(inp, r=pos_enc, u=self.u, v=self.v, mask=mask, mem=None)
        core_out = inp[:,-x_len:]
        return core_out

In [19]:
from fastai.text.learner import _model_meta

In [20]:
config['mem_len'] = 0
config['mask'] = False
config['bs'] = 4

In [21]:
_model_meta[MusicTransformer] = _model_meta[TransformerXL]
_model_meta[MusicTransformer]['config_lm'] = config

In [22]:
config

{'ctx_len': 150,
 'n_layers': 16,
 'n_heads': 8,
 'd_model': 256,
 'd_head': 32,
 'd_inner': 2048,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.GeLU: 3>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 0,
 'mask': False,
 'pad_idx': 1,
 'bos_idx': 0,
 'sep_idx': 8,
 'transpose_range': (0, 12),
 'note_range': (9, 138),
 'bs': 4,
 'bptt': 256,
 'vocab_size': 274}

In [23]:
class BertHead(nn.Module):
    def __init__(self, encoder, mask_decoder, ns_decoder, s2s_decoder):
        super().__init__()
        self.encoder = encoder
        self.mask_decoder = mask_decoder
        self.ns_decoder = ns_decoder
        self.s2s_decoder = s2s_decoder
        
    def forward(self, x, y=None):
#         x_emb = self.embed(x)
        x_enc = self.encoder(x)
        
        if y is None: # mask, and next sentence task
            return self.mask_decoder(x_enc), self.ns_decoder(x_enc)
        
        return self.mask_decoder(x_enc), self.s2s_decoder(x_enc, y)
    
    def __getitem__(self, idx):
        return [self.encoder, self.mask_decoder, self.ns_decoder, self.s2s_decoder][idx]
        
    "A sequential module that passes the reset call to its children."
    def reset(self): pass

In [24]:
def window_mask(x_len, device, m_len=0, size=(1,1)):
    win_size,k = size
    mem_mask = np.zeros((x_len,m_len))
    tri_mask = np.triu(np.ones((x_len//win_size+1,x_len//win_size+1)),k=k)
    window_mask = tri_mask.repeat(win_size,axis=0).repeat(win_size,axis=1)[:x_len,:x_len]
    np_mask = np.concatenate((mem_mask, window_mask), axis=1)
    mask = torch.tensor(np_mask, device=device).byte()[None,None]
    return mask
    
def rand_window_mask(x_len,m_len,device,max_size=3,p=0.2,is_eval=False):
    if is_eval or m_len == 0 or np.random.rand() >= p: 
        win_size,k = (1,1)
    else: win_size,k = (np.random.randint(0,max_size)+1,0)
    return window_mask(x_len, device, m_len, size=(win_size,k))

In [25]:
def get_output_mask(inp, pad_idx:int=1):
    return torch.triu(inp.new_ones(inp.size(1),inp.size(1)), diagonal=1)[None,None].byte()

In [26]:

class KVMultiHeadRelativeAttention(nn.Module):
    "MutiHeadAttention with relative positional encoding."
    def __init__(self, n_heads:int, d_model:int, d_head:int=None, resid_p:float=0., attn_p:float=0., bias:bool=True,
                 scale:bool=True):
        super().__init__()
        d_head = ifnone(d_head, d_model//n_heads)
        self.n_heads,self.d_head,self.scale = n_heads,d_head,scale
        
        self.q_wgt = nn.Linear(d_model, n_heads * d_head, bias=bias)
        self.k_wgt = nn.Linear(d_model, n_heads * d_head, bias=bias)
        self.v_wgt = nn.Linear(d_model, n_heads * d_head, bias=bias)
        
        self.out = nn.Linear(n_heads * d_head, d_model, bias=bias)
        self.drop_att,self.drop_res = nn.Dropout(attn_p),nn.Dropout(resid_p)
        self.ln = nn.LayerNorm(d_model)
        self.r_attn = nn.Linear(d_model, n_heads * d_head, bias=bias)
        
    def forward(self, q:Tensor, k:Tensor, v:Tensor, 
                r:Tensor=None, g_u:Tensor=None, g_v:Tensor=None, 
                mask:Tensor=None, **kwargs):
        return self.ln(q + self.drop_res(self.out(self._apply_attention(q, k, v, r, g_u, g_v, mask=mask, **kwargs))))
    
    def _apply_attention(self, q:Tensor, k:Tensor, v:Tensor, 
                         r:Tensor=None, g_u:Tensor=None, g_v:Tensor=None, 
                         mask:Tensor=None):
        #Notations from the paper: x input, r vector of relative distance between two elements, u et v learnable
        #parameters of the model common between all layers, mask to avoid cheating and mem the previous hidden states.
        bs,x_len,seq_len = q.size(0),q.size(1),r.size(0)
        wq,wk,wv = self.q_wgt(q),self.k_wgt(k),self.v_wgt(v)
        wq = wq[:,-x_len:]
        wq,wk,wv = map(lambda x:x.view(bs, x.size(1), self.n_heads, self.d_head), (wq,wk,wv))
        wq,wk,wv = wq.permute(0, 2, 1, 3),wk.permute(0, 2, 3, 1),wv.permute(0, 2, 1, 3)
        wkr = self.r_attn(r)
        wkr = wkr.view(seq_len, self.n_heads, self.d_head)
        wkr = wkr.permute(1,2,0)
        #### compute attention score (AC is (a) + (c) and BS is (b) + (d) in the paper)
        AC = torch.matmul(wq+g_u,wk)
        BD = _line_shift(torch.matmul(wq+g_v, wkr))
        if self.scale: attn_score = (AC + BD).mul_(1/(self.d_head ** 0.5))
        if mask is not None: 
            attn_score = attn_score.float().masked_fill(mask, -float('inf')).type_as(attn_score)
        attn_prob = self.drop_att(F.softmax(attn_score, dim=-1))
        attn_vec = torch.matmul(attn_prob, wv)
        return attn_vec.permute(0, 2, 1, 3).contiguous().view(bs, x_len, -1)

In [27]:
class S2SDecoderBlock(nn.Module):
    "Decoder block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, n_heads:int, d_model:int, d_head:int, d_inner:int, resid_p:float=0., attn_p:float=0., ff_p:float=0.,
                 bias:bool=True, scale:bool=True, double_drop:bool=True, **kwargs):
        super().__init__()
        self.mha1 = KVMultiHeadRelativeAttention(n_heads, d_model, d_head, resid_p=resid_p, attn_p=attn_p, bias=bias, scale=scale)
        self.mha2 = KVMultiHeadRelativeAttention(n_heads, d_model, d_head, resid_p=resid_p, attn_p=attn_p, bias=bias, scale=scale)
        self.ff   = feed_forward(d_model, d_inner, ff_p=ff_p, double_drop=double_drop)
    
    def forward(self, targ:Tensor, enc:Tensor, 
                r=None, u=None, v=None,
                mask_in:Tensor=None, mask_out:Tensor=None): 
        y = self.mha1(targ, targ, targ, r, u, v, mask=mask_out)
        return self.ff(self.mha2(y, enc, enc, r, u, v, mask=mask_in))

In [28]:
class S2SDecoder(nn.Module):
    def __init__(self, embed:nn.Module, n_hid:int, vocab_sz:int, n_layers:int, n_heads:int, d_model:int, d_head:int, d_inner:int, 
                 resid_p:float=0., attn_p:float=0., ff_p:float=0., bias:bool=False, scale:bool=True,
                 act:Activation=Activation.ReLU, double_drop:bool=True, attn_cls:Callable=MultiHeadRelativeAttention,
                 learned_pos_enc:bool=False, mask:bool=True, **kwargs):
        super().__init__()
        self.encoder = embed
        self.u = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.v = nn.Parameter(torch.Tensor(n_heads, 1, d_head)) #Remove 1 for einsum implementation of attention
        self.n_layers,self.d_model,self.mask = n_layers,d_model,mask
        self.layers = nn.ModuleList([S2SDecoderBlock(n_heads, d_model, d_head, d_inner, resid_p=resid_p, attn_p=attn_p,
                      ff_p=ff_p, bias=bias, scale=scale, act=act, double_drop=double_drop, 
                      attn_cls=attn_cls) for k in range(n_layers)])
        self.head = MusicLinearDecoder(d_model, vocab_sz, tie_encoder=embed.embed, **kwargs)
    
        nn.init.normal_(self.u, 0., 0.02)
        nn.init.normal_(self.v, 0., 0.02)
        
    def forward(self, enc, targ):
        # x = encoder, y = target
        bs,targ_len = targ.size()
        
        targ_emb, pos_enc = self.encoder(targ)

#         mask = window_mask(x_len, x.device) if self.mask else None
        mask_out = window_mask(targ_len, targ_emb.device)
        
        for i, layer in enumerate(self.layers):
            out = layer(targ_emb, enc, mask_out=mask_out,
                        r=pos_enc, u=self.u, v=self.v)
        return self.head(out)

In [29]:
# class S2SDecoder(nn.Module):
#     def __init__(self, embed, n_hid, vocab_sz, n_layers, **kwargs):
#         super().__init__()
#         self.decoder = nn.ModuleList([S2SDecoderBlock(**kwargs) for _ in range(n_layers)])
#         self.head = MusicLinearDecoder(n_hid, vocab_sz, tie_encoder=embed, **kwargs)
        
# #         self.pad_idx = pad_idx
        
#     def forward(self, inp, out):
#         x_len = inp.shape[-1]
# #         mask_out = rand_window_mask(x_len, 0, inp.device, is_eval=not self.training)
#         mask_out = window_mask(x_len, inp.device)
    
#         out = self.embed(out)
#         for dec_block in self.decoder: out = dec_block(out, inp, mask_in, mask_out)
#         return self.head(out)


In [30]:

class MusicLinearDecoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_hid:int, n_out:int, output_p:float, tie_encoder:nn.Module=None, bias:bool=True, **kwargs):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        output = self.output_dp(input)
        decoded = self.decoder(output)
        return decoded


In [31]:
def get_music_model(vocab_sz:int, config:dict=None, drop_mult:float=1.):
    "Create a language model from `arch` and its `config`, maybe `pretrained`."
    for k in config.keys(): 
        if k.endswith('_p'): config[k] *= drop_mult
#     tie_weights,output_p,out_bias = map(config.pop, ['tie_weights', 'output_p', 'out_bias'])
    tie_weights,output_p,out_bias = map(config.get, ['tie_weights', 'output_p', 'out_bias'])
    n_hid = config['d_model']
    embed = TransformerEmbedding(vocab_sz, n_hid, inp_p=config['embed_p'])
    encoder = MusicTransformer(embed=embed, **config)
    mask_decoder = MusicLinearDecoder(n_hid, vocab_sz, output_p, tie_encoder=embed.embed, bias=out_bias)
    ns_decoder = MusicLinearDecoder(n_hid, 4, output_p, tie_encoder=None, bias=out_bias)
    s2s_decoder = S2SDecoder(embed, n_hid, vocab_sz, **config)
    model = BertHead(encoder, mask_decoder, ns_decoder, s2s_decoder)
    return model.apply(init_transformer)


def music_model_learner(data:DataBunch, config:dict=None, drop_mult:float=1., pretrained:bool=False,
                        pretrained_fnames:OptStrTuple=None, **learn_kwargs) -> 'LanguageLearner':
    "Create a `Learner` with a language model from `data` and `arch`."
    model = get_music_model(config['vocab_size'], config=config, drop_mult=drop_mult)
    
    meta = _model_meta[TransformerXL]
    learn = MusicLearner(data, model, split_func=meta['split_lm'], 
                         bos_idx=config['bos_idx'], sep_idx=config['sep_idx'],
                        **learn_kwargs)
    
    if pretrained:
        if 'url' not in meta: 
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta['url'], data=False)
        fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        learn.load_pretrained(*fnames)
        learn.freeze()
    if pretrained_fnames is not None:
        fnames = [learn.path/learn.model_dir/f'{fn}.{ext}' for fn,ext in zip(pretrained_fnames, ['pth', 'pkl'])]
        learn.load_pretrained(*fnames)
        learn.freeze()
    return learn

## Load

In [32]:
learn = music_model_learner(data, config.copy())

Sep_idx: 8


In [33]:
xb,yb = data.one_batch(cpu=False)

In [34]:
# xb.shape, yb.shape

In [35]:
class BertLoss():
    def __init__(self, mask_loss, sent_loss):
        self.mask_loss = mask_loss
        self.sent_loss = sent_loss
        
    def __call__(self, input:Tensor, target:Tensor, target_sen:Tensor, **kwargs)->Rank0Tensor:
        m = self.mask_loss.__call__(input[0], target, **kwargs)
        s = self.sent_loss.__call__(input[1], target_sen, **kwargs)
        return m + s

In [36]:
class BertTrainer(LearnerCallback):
    "`Callback` that regroups lr adjustment to seq_len, AR and TAR."
    def __init__(self, learn:Learner):
        super().__init__(learn)
        
#     def on_loss_begin(self, last_output:Tuple[Tensor,Tensor,Tensor], **kwargs):
#         "Save the extra outputs for later and only returns the true output."
#         return {'last_output': (last_output[0], last_output[1]) }


In [37]:
# learn.callbacks = [BertTrainer(learn, alpha=2, beta=1)]
learn.callbacks = [BertTrainer(learn)]

In [38]:
learn.loss_func = BertLoss(CrossEntropyFlat(ignore_index=vocab.pad_idx), CrossEntropyFlat())

In [41]:
learn.model(xb, yb[0])

(tensor([[[-0.0743,  0.3196,  0.3367,  ...,  0.2344,  0.2848, -0.0412],
          [ 0.3629,  0.2655,  0.1444,  ...,  0.0260,  0.1932, -0.5172],
          [-0.0354, -0.1314, -0.3341,  ...,  0.1010,  0.1620, -0.1120],
          ...,
          [-0.2748,  0.2047,  0.2082,  ...,  0.2025,  0.2456,  0.6037],
          [-0.0886,  0.0361,  0.3315,  ...,  0.2643,  0.4652,  0.4201],
          [ 0.1788, -0.2791,  0.1327,  ...,  0.0840,  0.3184,  0.0697]],
 
         [[ 0.4849, -0.0831,  0.0960,  ..., -0.1094,  0.6874,  0.0190],
          [ 0.1493, -0.2390,  0.0464,  ..., -0.0780,  0.7060, -0.0755],
          [ 0.3525, -0.1121,  0.0144,  ...,  0.4640,  0.1898,  0.1246],
          ...,
          [-0.0620,  0.1672,  0.0721,  ...,  0.2152,  0.3309,  0.4012],
          [ 0.6216, -0.0695, -0.1472,  ..., -0.3827,  0.2661, -0.1737],
          [ 0.3156,  0.0521,  0.3747,  ..., -0.1284,  0.4987,  0.0275]],
 
         [[ 0.3770,  0.4130, -0.2621,  ..., -0.3210,  0.1762, -0.0058],
          [ 0.1668,  0.3123,

In [42]:
def mask_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    n = t1.shape[0]
    input = input[0].argmax(dim=-1).view(n,-1)
    t1 = t1.view(n,-1)
    mask = t1 != vocab.pad_idx
    return (input[mask]==t1[mask]).float().mean()

def ns_acc(input:Tensor, t1:Tensor, t2:Tensor)->Rank0Tensor:
    return accuracy(input[1], t2[0])

In [40]:
learn.metrics = [mask_acc, ns_acc]

In [41]:
learn.validate()

[6.7785196, tensor(0.1001), tensor(0.3444)]

In [42]:
# learn.lr_find()
# learn.recorder.plot()

In [43]:
learn.fit_one_cycle(3, 1e-4)

epoch,train_loss,valid_loss,mask_acc,ns_acc,time
0,3.288463,3.077294,0.539648,0.576416,02:03
1,2.314284,2.585530,0.588401,0.324031,02:04
2,2.121686,2.169067,0.596297,0.499136,02:04
